<a href="https://colab.research.google.com/github/Anushka091922/Jet-tagging-with-convolutional-neural-networks-Particle-Physics-/blob/main/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 3 - Neural network deep dive

## Setup

In [ ]:
# Uncomment and run this cell if using Colab, Kaggle etc
%pip install fastai==2.6.0 datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Imports

In [ ]:
import math

import torch
from datasets import load_dataset
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

In [ ]:
import datasets

# Suppress logs to keep things tidy
datasets.logging.set_verbosity_error()

## The dataset

In [ ]:
nsubjet_ds = load_dataset("dl4phys/top_tagging_nsubjettiness")
df = nsubjet_ds["train"].to_pandas()
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

,pT,mass,tau_1_0.5,tau_1_1,tau_1_2,tau_2_0.5,tau_2_1,tau_2_2,tau_3_0.5,tau_3_1,...,tau_4_0.5,tau_4_1,tau_4_2,tau_5_0.5,tau_5_1,tau_5_2,tau_6_0.5,tau_6_1,tau_6_2,label
0,543.633944,25.846792,0.165122,0.032661,0.002262,0.048830,0.003711,0.000044,0.030994,0.001630,...,0.024336,0.001115,0.000008,0.004252,0.000234,7.706005e-07,0.000000,0.000000,0.000000e+00,0
1,452.411860,13.388679,0.162938,0.027598,0.000876,0.095902,0.015461,0.000506,0.079750,0.009733,...,0.056854,0.005454,0.000072,0.044211,0.004430,6.175314e-05,0.037458,0.003396,3.670517e-05,0
2,429.495258,32.021091,0.244436,0.065901,0.005557,0.155202,0.038807,0.002762,0.123285,0.025339,...,0.078205,0.012678,0.000567,0.052374,0.005935,9.395772e-05,0.037572,0.002932,2.237277e-05,0
3,512.675443,6.684734,0.102580,0.011369,0.000170,0.086306,0.007760,0.000071,0.068169,0.005386,...,0.044705,0.002376,0.000008,0.027895,0.001364,4.400042e-06,0.009012,0.000379,6.731099e-07,0
4,527.956859,133.985415,0.407009,0.191839,0.065169,0.291460,0.105479,0.029753,0.209341,0.049187,...,0.143768,0.033249,0.003689,0.135407,0.029054,2.593460e-03,0.110805,0.023179,2.202088e-03,0


### Preparing the data

In [ ]:
train_df, valid_df = train_test_split(df, random_state=42)
train_df.shape, valid_df.shape

((908250, 21), (302750, 21))

In [ ]:
train_df.describe()

,pT,mass,tau_1_0.5,tau_1_1,tau_1_2,tau_2_0.5,tau_2_1,tau_2_2,tau_3_0.5,tau_3_1,...,tau_4_0.5,tau_4_1,tau_4_2,tau_5_0.5,tau_5_1,tau_5_2,tau_6_0.5,tau_6_1,tau_6_2,label
count,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,...,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000,908250.000000
mean,487.107393,88.090520,0.366716,0.198446,0.319559,0.222759,0.079243,0.072535,0.148137,0.035372,...,0.112024,0.022150,0.008670,0.088400,0.015329,0.004875,0.070679,0.011019,0.002914,0.500366
std,48.568267,48.393646,0.186922,0.339542,2.003898,0.110955,0.125155,0.674091,0.072627,0.051869,...,0.059393,0.032004,0.155468,0.051949,0.022866,0.107641,0.046571,0.017133,0.078247,0.500000
min,225.490387,-0.433573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,452.879289,39.958178,0.224456,0.058381,0.006443,0.139269,0.025638,0.001565,0.094603,0.013308,...,0.069037,0.007949,0.000188,0.051012,0.004936,0.000079,0.036142,0.002977,0.000033,0.000000
50%,485.894050,99.887418,0.380172,0.166016,0.045887,0.222763,0.061597,0.008788,0.148810,0.028501,...,0.110220,0.017609,0.000787,0.086045,0.011755,0.000387,0.067797,0.008028,0.000193,1.000000
75%,520.506446,126.518545,0.477122,0.240550,0.074417,0.299708,0.108207,0.022441,0.196156,0.046588,...,0.151137,0.029990,0.002006,0.121905,0.021089,0.001103,0.100437,0.015359,0.000635,1.000000
max,647.493145,299.211555,2.431888,6.013309,37.702422,2.218956,5.392683,33.352249,1.917912,4.502011,...,1.616280,3.753716,21.161948,1.407356,3.158352,17.645603,1.388879,3.127371,17.340970,1.000000


In [ ]:
# Slice out all feature columns
train_x = train_df.iloc[:, :-1].values
# Slice out the label column
train_y = train_df.iloc[:, -1].values

Next, we use the `MinMaxScaler` from scikit-learn to apply the normalization on the features array:

In [ ]:
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
# Sanity check the normalization worked
np.min(train_x), np.max(train_x)

(0.0, 1.0)

 let's convert these NumPy arrays to PyTorch tensors. PyTorch provides a handy `from_numpy()` method that allows us to do the conversion easily:

In [ ]:
# Cast to float32
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_df.iloc[:, -1].values)
# Sanity check on the shapes
train_x.shape, train_y.shape

(torch.Size([908250, 20]), torch.Size([908250]))

## Logistic regression as a neural network

In [ ]:
set_seed(42)
# Xavier initialisation
weights = torch.randn(20, 2) / math.sqrt(20)
# Track grads after initialization
weights.requires_grad_()
bias = torch.zeros(2, requires_grad=True)

In [ ]:
x = torch.tensor([1000.0, 1000.0, 1000.0])
x.exp()

tensor([inf, inf, inf])

In [ ]:
x = torch.tensor([-1000.0, -1000.0, -1000.0])
x.exp()

tensor([0., 0., 0.])

In [ ]:
def log_softmax(x):
    return (x - x.max()) - (x - x.max()).exp().sum(-1).log().unsqueeze(-1)


log_softmax(x)

tensor([-1.0986, -1.0986, -1.0986])

we now have an activation function that is numerically stable. Let's now define our logistic regression model to take a mini-batch `xb` of inputs and output the log-softmax values:

In [ ]:
def model(xb):
    return log_softmax(xb @ weights + bias)

here we will test the model with a batch of data from our training set (also called a _forward pass_):

In [ ]:
# Batch size
bs = 1024
# A mini-batch from x
xb = train_x[0:bs]
# Model predictions
preds = model(xb)
preds[0], preds.shape

(tensor([-0.5103, -0.9171], grad_fn=<SelectBackward0>), torch.Size([1024, 2]))

At this state the predictions are random, since we started with random weights. To improve these values, the next thing we need is a loss function. For classification tasks, one computes the _cross entropy_, which is the log likelihood of the softmax:

$$ {\cal L} = - \frac{1}{m} \sum_{i=1}^m \sum_{k=1}^K y_k^{(i)}\log\hat{p}_k^{(i)} \,.$$

However, we've already taken the log of the softmax values $\hat{p}_k^{(i)}$, so instead our loss will be the _negative log likelihood_, which doesn't include the logarithm.

In [ ]:
def nll_loss(predictions, target):
    # Mask predictions according to whether y_hat is 1 or 0
    return -predictions[range(target.shape[0]), target].mean()


loss_func = nll_loss

Here we have a loss function, we will test if we can compute the loss by comparing our mini-batch of predictions against a mini-batch of target values:

In [ ]:
yb = train_y[0:bs]
print(loss_func(preds, yb))

tensor(0.7619, grad_fn=<NegBackward0>)


Again, the loss value is random, but we can minimise this function with backpropagation. Before doing that, let's also compute the accuracy of the model so that we track progress during training:

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()


accuracy(preds, yb)

tensor(0.5020)

In [ ]:
# Learning rate
lr = 1e-2
# Number of epochs
epochs = 3
n = len(train_df)

for epoch in tqdm(range(epochs), desc="num_epochs"):
    for i in tqdm(range((n - 1) // bs + 1), leave=False):
        # 1. Select mini-batch
        start_i = i * bs
        end_i = start_i + bs
        xb = train_x[start_i:end_i]
        yb = train_y[start_i:end_i]
        # 2. Generate predictions
        pred = model(xb)
        # 3. Compute the loss
        loss = loss_func(pred, yb)
        # 4. Compute the gradients
        loss.backward()
        # 5. Update the weights and biases
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            # Set current gradients to zero
            weights.grad.zero_()
            bias.grad.zero_()

num_epochs:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

 here we update the weights and biases within the `torch.no_grad()` context manager - that's because we don't want these updates to be recorded in the tensors in the next iteration of gradient descent. We also set the gradients to zero after the update to prevent tracking these operations with every iteration.

so as we can see we've trained our model, let's compute the loss and accuracy to see if they've improved:

In [ ]:
def print_scores():
    print(f"Loss: {loss_func(model(xb), yb):.3f}")
    print(f"Accuracy: {accuracy(model(xb), yb):.3f}")

In [ ]:
print_scores()

Loss: 0.561
Accuracy: 0.857


## Refactoring with PyTorch's functional API

In [ ]:
loss_func = F.cross_entropy


def model(xb):
    return xb @ weights + bias


# Sanity check we get the same scores as before
print_scores()

Loss: 0.561
Accuracy: 0.857


## Refactoring with PyTorch's `nn` classes

here we will simplify our training loop by using the `nn.Module` and `nn.Parameter` classes. The first holds the weights and biases of the model, and defines the forward pass. The second, makes it simpler to keep track of the gradients:

In [ ]:
class LogisticRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(20, 2) / math.sqrt(20))
        self.bias = nn.Parameter(torch.zeros(2))

    def forward(self, xb):
        return xb @ self.weights + self.bias

Now when we instantiate this class, we get a newly initialized model, which we can generate predictions from, compute the loss etc:

In [ ]:
model = LogisticRegressor()
loss_func(model(xb), yb)

tensor(0.7114, grad_fn=<NllLossBackward0>)

In [ ]:
def fit():
    for epoch in tqdm(range(epochs), desc="num_epochs"):
        for i in tqdm(range((n - 1) // bs + 1), leave=False):
            # 1. Select mini-batch
            start_i = i * bs
            end_i = start_i + bs
            xb = train_x[start_i:end_i]
            yb = train_y[start_i:end_i]
            # 2. Generate predictions
            pred = model(xb)
            # 3. Compute the loss
            loss = loss_func(pred, yb)
            # 4. Compute the gradients
            loss.backward()
            # 5. Update the weights and biases
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()


fit()
print_scores()

num_epochs:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

Loss: 0.547
Accuracy: 0.859


In [ ]:
class LogisticRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(20, 2)

    def forward(self, xb):
        return self.linear(xb)


model = LogisticRegressor()
fit()
print_scores()

num_epochs:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

Loss: 0.530
Accuracy: 0.859


It works!

### Refactoring with PyTorch optimizers

In [ ]:
def get_model():
    model = LogisticRegressor()
    return model, torch.optim.SGD(model.parameters(), lr=lr)


model, optimizer = get_model()
loss_func(model(xb), yb)

tensor(0.7041, grad_fn=<NllLossBackward0>)

Now that we have a model and optimizer, we can refactor our `fit()` function as follows:

In [ ]:
def fit():
    for epoch in tqdm(range(epochs), desc="num_epochs"):
        for i in tqdm(range((n - 1) // bs + 1), leave=False):
            # 1. Select mini-batch
            start_i = i * bs
            end_i = start_i + bs
            xb = train_x[start_i:end_i]
            yb = train_y[start_i:end_i]
            # 2. Generate predictions
            pred = model(xb)
            # 3. Compute the loss
            loss = loss_func(pred, yb)
            # 4. Compute the gradients
            loss.backward()
            # 5. Update the weights and biases
            optimizer.step()
            optimizer.zero_grad()


fit()
print_scores()

num_epochs:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

Loss: 0.539
Accuracy: 0.860


## Refactoring with `Dataset` classes

In [ ]:
train_ds = TensorDataset(train_x, train_y)

This dataset has a length:

In [ ]:
len(train_ds)

908250

and we can index into it like a Python list:

In [ ]:
train_ds[0]

(tensor([6.1467e-01, 3.4553e-01, 2.0827e-01, 6.1930e-02, 2.9217e-02, 1.1426e-01,
         3.9722e-02, 3.0127e-02, 1.1553e-01, 4.4638e-02, 3.7694e-02, 1.1224e-01,
         5.0610e-02, 4.7183e-02, 7.6042e-02, 4.5295e-03, 2.0119e-05, 5.5926e-02,
         2.8786e-03, 8.6186e-06]),
 tensor(1))

Note that each element returns a _tuple_ of the feature and corresponding label. This means we can replace the mini-batch step to a single line of code:

```python
xb, yb = train_ds[i * bs : i * bs + bs]
```

Let's refactor our `fit()` function to use the `train_ds` object now:

In [ ]:
model, optimizer = get_model()


def fit():
    for epoch in tqdm(range(epochs), desc="num_epochs"):
        for i in tqdm(range((n - 1) // bs + 1), leave=False):
            # 1. Select mini-batch
            xb, yb = train_ds[i * bs : i * bs + bs]
            # 2. Generate predictions
            pred = model(xb)
            # 3. Compute the loss
            loss = loss_func(pred, yb)
            # 4. Compute the gradients
            loss.backward()
            # 5. Update the weights and biases
            optimizer.step()
            optimizer.zero_grad()


fit()
print_scores()

num_epochs:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

Loss: 0.541
Accuracy: 0.857


### Refactoring with DataLoaders

In [ ]:
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
next(iter(train_dl))

[tensor([[6.3268e-01, 7.1266e-02, 6.7265e-02,  ..., 1.5679e-02, 5.9481e-04,
          1.0084e-06],
         [5.5470e-01, 3.2794e-01, 1.4114e-01,  ..., 3.8751e-02, 2.0627e-03,
          1.1118e-05],
         [5.8864e-01, 4.2167e-01, 1.9011e-01,  ..., 4.6003e-02, 2.2277e-03,
          5.5698e-06],
         ...,
         [7.7654e-01, 4.6069e-01, 1.9646e-01,  ..., 5.1050e-02, 2.0193e-03,
          4.1955e-06],
         [5.1658e-01, 5.9622e-02, 5.8521e-02,  ..., 5.2730e-02, 2.2624e-03,
          6.7991e-06],
         [8.5039e-01, 3.4621e-02, 4.1556e-02,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 tensor([0, 1, 1,  ..., 1, 0, 0])]

We can then simply iterate over the `DataLoader` to get our mini-batches for the model:

In [ ]:
model, optimizer = get_model()


def fit():
    for epoch in tqdm(range(epochs), desc="num_epochs"):
        # 1. Select mini-batch
        for xb, yb in tqdm(train_dl, leave=False):
            # 2. Generate predictions
            pred = model(xb)
            # 3. Compute the loss
            loss = loss_func(pred, yb)
            # 4. Compute the gradients
            loss.backward()
            # 5. Update the weights and biases
            optimizer.step()
            optimizer.zero_grad()


fit()
print_scores()

num_epochs:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

Loss: 0.541
Accuracy: 0.861


## Going deeper

![](https://github.com/lewtun/dl4phys/blob/main/images/dropout.gif?raw=1)

In [ ]:
model = nn.Sequential(
    nn.Linear(20, 200),
    nn.ReLU(),
    nn.Linear(200, 200),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(200, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(50, 2),
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

fit()
print_scores()

num_epochs:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?it/s]

Loss: 0.262
Accuracy: 0.883


Not bad, we've got a decent boost from using a deeper model and better optimizer!

## Wrapping everything in a Learner

In [ ]:
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

# Slice out all feature columns and cast to float32
valid_x = valid_df.iloc[:, :-1].values
valid_x = scaler.fit_transform(valid_x)
valid_x = torch.from_numpy(valid_x).float()
# Slice out the label column
valid_y = torch.from_numpy(valid_df.iloc[:, -1].values)
# Create dataset and dataloader for validation set
valid_ds = TensorDataset(valid_x, valid_y)
valid_dl = DataLoader(valid_ds, batch_size=bs)

Now that we have dataloaders, recall that fastai wraps them in a single `DataLoaders` object:

In [ ]:
dls = DataLoaders(train_dl, valid_dl)

The final step is to define the model and optimizer:

In [ ]:
model = nn.Sequential(
    nn.Linear(20, 200),
    nn.ReLU(),
    nn.Linear(200, 200),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(200, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(50, 2),
)

opt_func = Adam

and wrap everything in a `Learner` and train for 3 epochs:

In [ ]:
learn = Learner(dls, model, loss_func, opt_func=opt_func, metrics=[accuracy])

In [ ]:
learn.fit(3, lr)

epoch,train_loss,valid_loss,accuracy,time
0,0.251602,0.350905,0.830299,00:28
1,0.244106,0.324598,0.844773,00:35
2,0.239266,0.314358,0.842031,00:30
